### Dependencies

In [1]:
!pip install matplotlib tensorflow tensorflow_addons tensorflow_datasets imageio

### Setup

In [2]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from tensorflow.keras import Model
import tensorflow as tf

from IPython import display

### Dataset

In [4]:
train_ds = tf.keras.utils.image_dataset_from_directory(
#   "/home/tony/TO_BE_REMOVED/celeba_data/imgs/",
  "/Users/anthonylaw/Desktop/Endless/GAN-devel/mnist_ds/mnist_jpg/training",
  seed=123,
  image_size=(32, 32),
  batch_size=16)

Found 60000 files belonging to 10 classes.


2022-08-28 13:44:27.121007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# for i_b, l_b in train_ds:
#     print(i_b.shape)
#     print(tf.image.rgb_to_grayscale(i_b).shape)

#### Note: 

Images should be normalized to [-1,1] ***(Done in Arch)***

### Generator network

#### Note:

Modify the network size for deployment

In [6]:
class Generator(Model):

    def __init__(self, noise_dim, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.noise_dim = noise_dim
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = int(min(max(min(image_shape[0]/8.0-3.0, image_shape[1]/8.0-3.0), 3.0), 9.0))

        self.lr_d = layers.ReLU()
        self.lr_c1 = layers.ReLU()
        self.lr_c2 = layers.ReLU()
        self.lr_c3 = layers.ReLU()
        
        self.init_dense = layers.Dense(image_shape[0]/8.0*image_shape[1]/8.0*64,
                               use_bias=False, input_shape=(self.noise_dim,))
        
        self.init_reshape = layers.Reshape((int(image_shape[0]/8.0), int(image_shape[1]/8.0), 64))
        
        self.conv2dT1 = layers.Conv2DTranspose(64, (self.kernel_size, self.kernel_size),
                                               strides=(1, 1), padding='same')
        self.conv2dT2 = layers.Conv2DTranspose(32, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2dT3 = layers.Conv2DTranspose(16, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2dTactv = layers.Conv2DTranspose(self.num_channel, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same', activation='tanh')

    def call(self, noise_vec):

        init_vec = tf.squeeze(self.lr_d(self.init_dense(noise_vec)))
        
#         print(init_vec.shape)
        
        reshaped = self.init_reshape(init_vec)
        
#         print(reshaped.shape)
        
        convt1 = self.lr_c1(self.conv2dT1(reshaped))

#         print(convt1.shape)
        
        convt2 = self.lr_c2(self.conv2dT2(convt1))
        
#         print(convt2.shape)
                         
        convt3 = self.lr_c3(self.conv2dT3(convt2))
        
#         print(convt3.shape)
            
        out = self.conv2dTactv(convt3)
        
#         print(out.shape)

        return out

#### Testing

In [7]:
g1 = Generator(10, (32, 32), 3)

In [8]:
g1.kernel_size

3

In [9]:
noise_input = tf.random.normal((5, 10))
print(noise_input.shape)
pics1 = g1(tf.expand_dims(noise_input, 0))
print(pics1.shape)
# plt.imshow(pics1[-1, :, :, :], cmap='gray')

(5, 10)
(5, 32, 32, 3)


In [10]:
g1.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 re_lu (ReLU)                multiple                  0         
                                                                 
 re_lu_1 (ReLU)              multiple                  0         
                                                                 
 re_lu_2 (ReLU)              multiple                  0         
                                                                 
 re_lu_3 (ReLU)              multiple                  0         
                                                                 
 dense (Dense)               multiple                  10240     
                                                                 
 reshape (Reshape)           multiple                  0         
                                                                 
 conv2d_transpose (Conv2DTra  multiple                 36

### Discriminator

In [11]:
class Discriminator(Model):

    def __init__(self, image_shape, num_channel):
        super().__init__()
        
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.kernel_size = int(min(max(min(image_shape[0]/8.0-3.0, image_shape[1]/8.0-3.0), 3.0), 9.0))

        self.lr_c1 = layers.LeakyReLU()
        self.lr_c2 = layers.LeakyReLU()
        self.lr_c3 = layers.LeakyReLU()
        self.flatten = layers.Flatten()
        
        self.conv2d1 = layers.Conv2D(16, (self.kernel_size, self.kernel_size),
                                        strides=(2, 2), padding='same',
                                        input_shape=(None, self.image_shape[0],
                                        self.image_shape[1], self.num_channel))
        self.conv2d2 = layers.Conv2D(32, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.conv2d3 = layers.Conv2D(64, (self.kernel_size, self.kernel_size),
                                               strides=(2, 2), padding='same')
        self.dense_actv = layers.Dense(64,
                                      )
#                                        activation="sigmoid")
        
    def call(self, img_input):
        
        conv1 = self.lr_c1(self.conv2d1(img_input))
        
#         print(conv1.shape)
        
        conv2 = self.lr_c2(self.conv2d2(conv1))
        
#         print(conv2.shape)
                         
        conv3 = self.lr_c3(self.conv2d3(conv2))
        
#         print(conv3.shape)
        
        flat = self.flatten(conv3)
        
#         print(flat.shape)
        
        out = tf.squeeze(self.dense_actv(flat))
        
#         print(out.shape)

        return out

#### Testing

In [12]:
d1 = Discriminator((32,32), 3)
g2 = Generator(10, (32,32), 3)
d1.kernel_size

3

In [13]:
noise_input = tf.random.normal((5, 10))
pics2 = g1(tf.expand_dims(noise_input, 0))
# plt.imshow(pics2[-1, :, :, 0], cmap='gray')
print(pics2.shape)

(5, 32, 32, 3)


In [14]:
deci = d1(pics2)
deci

<tf.Tensor: shape=(5, 64), dtype=float32, numpy=
array([[ 6.85174833e-04, -1.97368488e-03, -1.51406229e-03,
        -6.73288596e-04, -1.64995389e-03,  1.10560225e-03,
         4.90562874e-04, -3.07230977e-04,  1.27579924e-03,
        -1.04062399e-03, -7.33286259e-04,  6.21622952e-04,
        -1.44893397e-03,  5.56204002e-04,  8.09336489e-05,
         1.42288918e-05,  1.18794863e-03, -4.48025879e-04,
         5.87049581e-04,  5.14148967e-04, -5.99993858e-04,
        -7.24156387e-04, -8.08689860e-04, -1.25736557e-03,
        -1.67869875e-04, -2.54387781e-03,  2.05661752e-03,
         2.41507310e-03,  6.38983620e-04, -2.09220088e-04,
         1.11503317e-03, -1.18578109e-03,  3.43304710e-04,
         3.34557291e-04,  9.66406369e-04, -6.11025607e-05,
         8.40763823e-05, -1.15453277e-03,  3.62648163e-04,
        -9.63857980e-04, -2.89332820e-04, -1.27725059e-03,
         5.05678690e-05,  1.43870187e-04,  2.10751407e-03,
        -9.41049308e-04,  2.46559284e-05,  9.13130352e-05,
       

In [15]:
d1.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 leaky_re_lu (LeakyReLU)     multiple                  0         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   multiple                  0         
                                                                 
 leaky_re_lu_2 (LeakyReLU)   multiple                  0         
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 conv2d (Conv2D)             multiple                  448       
                                                                 
 conv2d_1 (Conv2D)           multiple                  4640      
                                                                 
 conv2d_2 (Conv2D)           multiple                

### CramerDCGAN

In [25]:
EPSILON = 1e-16

class DCGAN:
    
    def __init__(self, dataset_path, image_shape, num_channel, noise_latent_dim,
                 buffer_size=60000, batch_size=256, lr=3e-4, gp_lam = 10.0):
        assert len(image_shape) == 2
        assert image_shape[0]%8 == 0
        assert image_shape[1]%8 == 0
        
        self.image_shape = image_shape
        self.num_channel = num_channel
        self.noise_latent_dim = noise_latent_dim
        self.buffer_cap, self.batch_size, self.gp_lam = buffer_size, batch_size, gp_lam
        self.num_img_prog_monit = 16
        
        self.dataset = tf.keras.utils.image_dataset_from_directory(
                              dataset_path,
                              seed=123,
                              image_size=self.image_shape,
                              batch_size=self.batch_size)
        # NOTE: Dataset must be processed differently for different source and applications
        
        self.g = Generator(self.noise_latent_dim, self.image_shape, self.num_channel)
        self.d = Discriminator(self.image_shape, self.num_channel)
        
        self.g_opt = tf.keras.optimizers.Adam(lr)
        self.d_opt = tf.keras.optimizers.Adam(lr)
        
        self.g_seed = tf.random.normal((self.num_img_prog_monit, self.noise_latent_dim))

    def cramer_loss(self, d_x_data, d_g_z_1, d_g_z_2, x_it):
        
        crit_r = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_x_data**2, axis = 1)+EPSILON))
        crit_g_1 = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_g_z_1, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_g_z_1**2, axis = 1)+EPSILON))
        
        L_srg = tf.math.add(crit_r, -crit_g_1)
        
        with tf.GradientTape() as t_gp:
            t_gp.watch(x_it)
            d_it = self.d(x_it)
            crit_it = tf.math.add(tf.math.sqrt(tf.reduce_sum(tf.math.add(d_it, -d_g_z_2)**2, axis = 1)+EPSILON),
                   -tf.math.sqrt(tf.reduce_sum(d_it**2, axis = 1)+EPSILON))
            
        gp_grad = t_gp.gradient(crit_it, x_it)
        l2n_gp = tf.math.sqrt(tf.reduce_sum(gp_grad**2, axis = [1,2,3])+EPSILON)
        
        # d_loss
        L_d = tf.reduce_mean(-L_srg + (self.gp_lam*((l2n_gp-1.0)**2)))

        l2nrg1 = tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_1)**2, axis = 1)+EPSILON)
        l2nrg2 = tf.math.sqrt(tf.reduce_sum(tf.math.add(d_x_data, -d_g_z_2)**2, axis = 1)+EPSILON)
        l2ng12 = tf.math.sqrt(tf.reduce_sum(tf.math.add(d_g_z_1, -d_g_z_2)**2, axis = 1)+EPSILON)

        # g_loss
        L_g = tf.reduce_mean(l2nrg1 + l2nrg2 - l2ng12)

        return L_g, L_d
        
    
    @tf.function
    def update(self, imgs):
        noise_input1 = tf.random.normal((imgs.shape[0], self.noise_latent_dim))
        noise_input2 = tf.random.normal((imgs.shape[0], self.noise_latent_dim))
        
        with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
            g_z_1 = self.g(noise_input1)
            g_z_2 = self.g(noise_input2)
            
            d_x_data = self.d(imgs)
            d_g_z_1 = self.d(g_z_1)
            d_g_z_2 = self.d(g_z_2)
            
            epsi = tf.random.uniform([imgs.shape[0], 1, 1, 1], 0.0, 1.0)
            x_it = tf.math.add(epsi*imgs, (1.0-epsi)*g_z_1)
            g_loss, d_loss = self.cramer_loss(d_x_data, d_g_z_1, d_g_z_2, x_it)
            
        grad_g = g_tape.gradient(g_loss, self.g.trainable_variables)
        grad_d = d_tape.gradient(d_loss, self.d.trainable_variables)
        
        self.g_opt.apply_gradients(zip(grad_g, self.g.trainable_variables))
        self.d_opt.apply_gradients(zip(grad_d, self.d.trainable_variables))
        return g_loss, d_loss
        
    def train(self, epochs=50, train_moni_path=None):
        for epo in range(epochs):
            for img_b, l_b in self.dataset:
                if self.num_channel == 1 and img_b.shape[-1] == 3:
                    img_b = tf.image.rgb_to_grayscale(img_b)
                norm_img_b = (img_b-127.5)/127.5
                g_l, d_l = self.update(norm_img_b)
            print("Generator Loss: ", g_l, ", Discriminator Loss: ",  d_l)
            
            if not train_moni_path == None:
                self.monitor_progress(epo, train_moni_path)
            
    def monitor_progress(self, epo, path):
        pics = self.g(self.g_seed)
        
        fig = plt.figure(figsize=(4,4))
        for i in range(pics.shape[0]):
            plt.subplot(4,4,i+1)
            if self.num_channel == 1:
                plt.imshow(pics[i,:,:,0], cmap='gray')
            else:   
                plt.imshow(tf.cast(tf.math.round(pics[i,:,:,:]*127.5+127.5), tf.int32))
            plt.axis('off')
            
        plt.savefig(path+'/image_{:04d}.png'.format(epo))
#         plt.savefig('/home/tony/TO_BE_REMOVED/imgs/image_{:04d}.png'.format(epo))
        # NEEDS to be changed for machines
        
        plt.close('all')
        
    def save_weights(self, g_path, d_path):
        self.g.save_weights(g_path)
        print("Saved generator weights", flush=True)
        self.d.save_weights(d_path)
        print("Saved discriminator weights", flush=True)
    def load_weights(self, g_path, d_path):
        try:
            self.g.load_weights(g_path)
            print("Loaded generator weights", flush=True)
            self.d.load_weights(d_path)
            print("Loaded discriminator weights", flush=True)
        except ValueError:
            print("ERROR: Please make sure weights are saved as .ckpt", flush=True)
    
    def generate_samples(self, num_sam, path):
        sam_seed = tf.random.normal((num_sam, self.noise_latent_dim))
        sam_pics = self.g(sam_seed)
        for i in range(sam_pics.shape[0]):
            if self.num_channel == 1:
                plt.imshow(sam_pics[i,:,:,0], cmap='gray')
            else:   
                plt.imshow(tf.cast(tf.math.round(pics[i,:,:,:]*127.5+127.5), tf.int32))
            plt.axis('off')
            plt.savefig(path+'/image_{:04d}.png'.format(i))
            plt.close('all')
            

#### Testing

In [26]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "/Users/anthonylaw/Desktop/Endless/GAN-devel/mnist_ds/mnist_jpg/training",
  seed=123,
  image_size=(32, 32),
  batch_size=256)

Found 60000 files belonging to 10 classes.


In [27]:
ds_path = "/Users/anthonylaw/Desktop/Endless/GAN-devel/mnist_ds/mnist_jpg/training"

In [28]:
dcgan1 = DCGAN(ds_path, (32, 32), 1, 25)

Found 60000 files belonging to 10 classes.


In [29]:
dcgan1.train(5,'./imgs')
dcgan1.save_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

Generator Loss:  tf.Tensor(20.169584, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-13.992974, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(11.07236, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-6.241216, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(11.480617, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-6.047583, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(10.570411, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-4.2436476, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(10.378007, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-4.3242736, shape=(), dtype=float32)
Saved generator weights
Saved discriminator weights


In [30]:
dcgan1.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

Loaded generator weights
Loaded discriminator weights


In [32]:
dcgan1.generate_samples(10, './samples')

In [33]:
dcgan2 = DCGAN(ds_path, (32, 32), 1, 25)

Found 60000 files belonging to 10 classes.


In [34]:
dcgan2.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

Loaded generator weights
Loaded discriminator weights


In [36]:
dcgan2.train(10, './imgs')

Generator Loss:  tf.Tensor(11.761238, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-4.6782985, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(10.948012, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-3.9494812, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(9.988838, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-3.4645674, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(11.518083, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-2.2646255, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(10.231049, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-2.9900744, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(10.958645, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-2.9179451, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(10.184042, shape=(), dtype=float32) , Discriminator Loss:  tf.Tensor(-2.4785311, shape=(), dtype=float32)
Generator Loss:  tf.Tensor(9.687585, shape=(), dtype=float32) , Discri

In [37]:
dcgan2.save_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

Saved generator weights
Saved discriminator weights


In [38]:
dcgan1.load_weights('./weights/g_test.ckpt', './weights/d_test.ckpt')

Loaded generator weights
Loaded discriminator weights


In [42]:
dcgan1.generate_samples(10, './samples')